v.d.10: Classificação se é fato gerador ou não e da categoria (SE, CI, CA) de uma só fez (4 categorias: NÃO, SE, CI, CA) e em 2 etapas, utilizando TF-IDF sem e com limpeza de dados, balanceamento, otimização de hiperparâmetros e cross validation, feature importance do algoritimo de SGDClassifier. Além disso, será utilizado o dataset ampliado e uma amostra na otimização de hiperparâmetros.

# Classificação de lançamentos contábeis

1) Descrição do problema: classificar a despesa pública natureza 339036 (outros serviços - pessoa física) com base no histórico da nota de empenho como fato gerador, ou não, das contribuições previdenciárias.

2) Descrição da solução: Construção de features com base no texto do histórico das notas de empenho, treinamento e teste para seleção do modelo de classificação com melhor métrica de desempenho.

3) Fonte de dados: Os dados das notas de empenho estão disponíveis em portais da transparência de diversos órgãos públicos, por exemplo,
https://www.governotransparente.com.br/acessoinfo/44529487/empenhoportipo. No caso, será utilizada uma base de dados rotulados a partir desses dados públicos.

4) Variáveis independentes: texto com o histórico da nota de empenho.

5) Variável dependente. Primeiramente, será classificado apenas como 0 (não é fato gerador) e 1 (é fato gerador). Posteriormente, a classificação incluirá a categoria do segurado: segurado empregado, contribuinte individual, contribuinte individual – condutor autônomo.

## 1. Carregamento dos dados

In [1]:
# Monta google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Carrega os pacotes necessários
import pandas as pd
import numpy as np

# Importa bibliotecas
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

# Configurações
pd.set_option('display.max_colwidth', None)

In [3]:
# Importa os dados
df_completo = pd.read_excel('/content/drive/My Drive/projeto classificacao de lancamento/dados/despesa liquidada - base completa.xlsx')
df_completo.head()

,Município,Data da Liquidação,Natureza da Despesa,Numero do Empenho,Descrição do Empenho,Fato Gerador,Categoria
0,Concórdia do Pará,2019-01-31,339036,04010020,"Locacao de imovel para funcionamento da casa de apoio da Equipe Tatico metropolitana de Concordia do Para, no periodo de 01 de Janeiro a 31 de Dezembro de 2019.",Não,NaN
1,Concórdia do Pará,2019-01-31,339036,04010019,"Locacao de imovel para funcionamento do arquivo de documentos inativos e deposito de materiais inserviveis do Municipio de Concordia do Para, no periodo de 01 de janeiro a 31 de dezembro de 2019.",Não,NaN
2,Concórdia do Pará,2019-01-30,339036,28010007,"servicos eventuais prestados para atender a Secretaria Municipal de Obras e Transportes, como Mecanico de veiculos da secretaria durante o mes de Janeiro de 2019.",Sim,CI
3,Concórdia do Pará,2019-01-24,339036,18010004,"servicos eventuais prestados como Auxiliar de Servicos Gerais para atender a Secretaria Municipal de Administracao, durante 12 dias do mes de Janeiro de 2019.",Sim,CI
4,Concórdia do Pará,2019-01-24,339036,18010005,"servicos eventuais prestados como Vigia para atender a Secretaria Municipal de Administracao, durante 10 dias do mes de Janeiro de 2019.",Sim,CI


In [4]:
# Verifica a quantidade de lançamentos por município
df_completo['Município'].value_counts()

,count
Município,
Cametá,21142
Portel,8372
Concórdia do Pará,3248
Baião,2482
Garrafão do Norte,889
São Miguel do Guamá,814
Laranjal do Jari,620


In [5]:
# Seleciona as variáveis de interesse
df = df_completo[['Descrição do Empenho', 'Fato Gerador', 'Categoria']]
df.columns = ['descricao', 'fato_gerador', 'categoria']
df.head()

,descricao,fato_gerador,categoria
0,"Locacao de imovel para funcionamento da casa de apoio da Equipe Tatico metropolitana de Concordia do Para, no periodo de 01 de Janeiro a 31 de Dezembro de 2019.",Não,NaN
1,"Locacao de imovel para funcionamento do arquivo de documentos inativos e deposito de materiais inserviveis do Municipio de Concordia do Para, no periodo de 01 de janeiro a 31 de dezembro de 2019.",Não,NaN
2,"servicos eventuais prestados para atender a Secretaria Municipal de Obras e Transportes, como Mecanico de veiculos da secretaria durante o mes de Janeiro de 2019.",Sim,CI
3,"servicos eventuais prestados como Auxiliar de Servicos Gerais para atender a Secretaria Municipal de Administracao, durante 12 dias do mes de Janeiro de 2019.",Sim,CI
4,"servicos eventuais prestados como Vigia para atender a Secretaria Municipal de Administracao, durante 10 dias do mes de Janeiro de 2019.",Sim,CI


In [6]:
df['fato_gerador'].value_counts()

,count
fato_gerador,
Sim,29694
Não,7873


In [7]:
df['categoria'].value_counts()

,count
categoria,
CI,28259
CA,1100
SE,335


In [8]:
print('Formato:', df.shape)
df.isnull().sum()

Formato: (37567, 3)


,0
descricao,0
fato_gerador,0
categoria,7873


In [9]:
# Completar os dados faltantes da coluna categoria com 'Não'
df['categoria'] = df['categoria'].fillna(value='Não')

<ipython-input-9-46103d48ae62>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['categoria'] = df['categoria'].fillna(value='Não')


In [10]:
print('Formato:', df.shape)
df.isnull().sum()

Formato: (37567, 3)


,0
descricao,0
fato_gerador,0
categoria,0


## 2. Pré-processamento do texto



In [11]:
# Importação das bibliotecas
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### 2.1. Stop words

In [12]:
# Stopwords da biblioteca nltk em português
stop_words_nltk = stopwords.words('portuguese')

# Meses
meses = ['janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']

# Stopwords específicas do corpus
corpus_stop_words = ['cameta', 'portel', 'laranjal', 'jari', 'garrafão', 'concordia', 'zona', 'rural', 'belém', 'pacaja', 'pacajar', 'anapu',
                     'aluno', 'mês', 'serviços', 'servicos', 'referente', 'prestados', 'prestados', 'pagamento', 'municipal', 'município', 'valor', 'empenha',
                     'favor', 'credor', 'acima', 'ocorrer', 'secretaria', 'ensino', 'mes', 'mês', 'durante', 'junto', 'atender', 'periodo', 'ref', 'm', '``']

stop_words = stop_words_nltk + meses + corpus_stop_words

In [13]:
print(stop_words)

['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvésse

In [14]:
lista = ['a', 'à', 'ao', 'aos', 'aquela', 'aquelas', 'aquele', 'aqueles', 'aquilo', 'as', 'às', 'até', 'com', 'como', 'da', 'das', 'de', 'dela', 'delas', 'dele', 'deles', 'depois', 'do', 'dos', 'e', 'é', 'ela', 'elas', 'ele', 'eles', 'em', 'entre', 'era', 'eram', 'éramos', 'essa', 'essas', 'esse', 'esses', 'esta', 'está', 'estamos', 'estão', 'estar', 'estas', 'estava', 'estavam', 'estávamos', 'este', 'esteja', 'estejam', 'estejamos', 'estes', 'esteve', 'estive', 'estivemos', 'estiver', 'estivera', 'estiveram', 'estivéramos', 'estiverem', 'estivermos', 'estivesse', 'estivessem', 'estivéssemos', 'estou', 'eu', 'foi', 'fomos', 'for', 'fora', 'foram', 'fôramos', 'forem', 'formos', 'fosse', 'fossem', 'fôssemos', 'fui', 'há', 'haja', 'hajam', 'hajamos', 'hão', 'havemos', 'haver', 'hei', 'houve', 'houvemos', 'houver', 'houvera', 'houverá', 'houveram', 'houvéramos', 'houverão', 'houverei', 'houverem', 'houveremos', 'houveria', 'houveriam', 'houveríamos', 'houvermos', 'houvesse', 'houvessem', 'houvéssemos', 'isso', 'isto', 'já', 'lhe', 'lhes', 'mais', 'mas', 'me', 'mesmo', 'meu', 'meus', 'minha', 'minhas', 'muito', 'na', 'não', 'nas', 'nem', 'no', 'nos', 'nós', 'nossa', 'nossas', 'nosso', 'nossos', 'num', 'numa', 'o', 'os', 'ou', 'para', 'pela', 'pelas', 'pelo', 'pelos', 'por', 'qual', 'quando', 'que', 'quem', 'são', 'se', 'seja', 'sejam', 'sejamos', 'sem', 'ser', 'será', 'serão', 'serei', 'seremos', 'seria', 'seriam', 'seríamos', 'seu', 'seus', 'só', 'somos', 'sou', 'sua', 'suas', 'também', 'te', 'tem', 'tém', 'temos', 'tenha', 'tenham', 'tenhamos', 'tenho', 'terá', 'terão', 'terei', 'teremos', 'teria', 'teriam', 'teríamos', 'teu', 'teus', 'teve', 'tinha', 'tinham', 'tínhamos', 'tive', 'tivemos', 'tiver', 'tivera', 'tiveram', 'tivéramos', 'tiverem', 'tivermos', 'tivesse', 'tivessem', 'tivéssemos', 'tu', 'tua', 'tuas', 'um', 'uma', 'você', 'vocês', 'vos', 'janeiro', 'fevereiro', 'março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro', 'cameta', 'portel', 'laranjal', 'jari', 'garrafão', 'concordia', 'zona', 'rural', 'belém', 'pacaja', 'pacajar', 'anapu', 'aluno', 'mês', 'serviços', 'servicos', 'referente', 'prestados', 'prestados', 'pagamento', 'municipal', 'município', 'valor', 'empenha', 'favor', 'credor', 'acima', 'ocorrer', 'secretaria', 'ensino', 'mes', 'mês', 'durante', 'junto', 'atender', 'periodo', 'ref', 'm', '``']

### 2.2. Limpa texto (remove pontuação, números, stopwords e transforma em letra minúscula).

In [ ]:
# Função para limpeza de texto
def limpa_texto(text, stop_words=[]):
    # Substitui sinais de pontuação por espaço em branco
    text = text.replace('.', ' ')
    text = text.replace(',', ' ')
    text = text.replace(';', ' ')
    text = text.replace(':', ' ')
    text = text.replace('!', ' ')
    text = text.replace('?', ' ')
    text = text.replace('(', ' ')
    text = text.replace(')', ' ')
    text = text.replace('[', ' ')
    text = text.replace(']', ' ')
    text = text.replace('{', ' ')
    text = text.replace('}', ' ')
    text = text.replace('/', ' ')
    text = text.replace('-', ' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = ''.join(word for word in text if not word.isdigit()).lower()         # remove números e coloca em minúscula
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words]                # stopwords
    return ' '.join(tokens)

In [ ]:
# Exemplo de aplicação
texto = 'Locacao de imovel para funcionamento da casa de apoio da Equipe Tatico metropolitana de Concordia do Para, no periodo de 01 de Janeiro a 31 de Dezembro de 2019'
texto_limpo = limpa_texto(texto, stop_words)
print('Texto original:', texto)
print('Texto limpo:', texto_limpo)

Texto original: Locacao de imovel para funcionamento da casa de apoio da Equipe Tatico metropolitana de Concordia do Para, no periodo de 01 de Janeiro a 31 de Dezembro de 2019
Texto limpo: locacao imovel funcionamento casa apoio equipe tatico metropolitana


### 2.3. Lematização
A lematização é uma técnica essencial no campo do Processamento de Linguagem Natural (NLP). Ela busca simplificar a análise textual ao normalizar palavras para sua forma base, conhecida como “lema”. Por exemplo, considere os substantivos “gato”, “gatinho” e “gatos”; todos eles compartilham do lema “gato”.

Fonte: https://medium.com/@guilherme.davedovicz/nlp-para-iniciantes-lematiza%C3%A7%C3%A3o-d3f723fa9ee3

In [ ]:
# Importação bibioteca, download e carregamento do modelo 'pt_core_news_sm'
import spacy
spacy.cli.download("pt_core_news_sm")
nlp = spacy.load('pt_core_news_sm')

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# Função de lematização com spaCy
def lematiza_spacy(texto):
    doc = nlp(texto)
    palavras_lemmatizadas = [token.lemma_ for token in doc]
    return ' '.join(palavras_lemmatizadas)

In [ ]:
# Exemplo de aplicação
texto_lematizado = lematiza_spacy(texto_limpo)
print('Texto limpo:', texto_limpo)
print('Texto lematizado:', texto_lematizado)

Texto limpo: locacao imovel funcionamento casa apoio equipe tatico metropolitana
Texto lematizado: locacao imovel funcionamento casa apoio equipe tatico metropolitana


In [ ]:
df['descricao_limpa'] = df['descricao'].apply(lambda x: limpa_texto(x, stop_words))
df['descricao_lematizada'] = df['descricao_limpa'].apply(lambda x: lematiza_spacy(x))

<ipython-input-18-6dbbdb96977c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['descricao_limpa'] = df['descricao'].apply(lambda x: limpa_texto(x, stop_words))
<ipython-input-18-6dbbdb96977c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['descricao_lematizada'] = df['descricao_limpa'].apply(lambda x: lematiza_spacy(x))


In [ ]:
df.head(20)

,descricao,fato_gerador,categoria,descricao_limpa,descricao_lematizada
0,"Locacao de imovel para funcionamento da casa de apoio da Equipe Tatico metropolitana de Concordia do Para, no periodo de 01 de Janeiro a 31 de Dezembro de 2019.",Não,Não,locacao imovel funcionamento casa apoio equipe tatico metropolitana,locacao imovel funcionamento casa apoio equipe tatico metropolitana
1,"Locacao de imovel para funcionamento do arquivo de documentos inativos e deposito de materiais inserviveis do Municipio de Concordia do Para, no periodo de 01 de janeiro a 31 de dezembro de 2019.",Não,Não,locacao imovel funcionamento arquivo documentos inativos deposito materiais inserviveis municipio,locacao imovel funcionamento arquivo documento inativo deposito material inservivei municipio
2,"servicos eventuais prestados para atender a Secretaria Municipal de Obras e Transportes, como Mecanico de veiculos da secretaria durante o mes de Janeiro de 2019.",Sim,CI,eventuais obras transportes mecanico veiculos,eventual obra transporte mecanico veiculo
3,"servicos eventuais prestados como Auxiliar de Servicos Gerais para atender a Secretaria Municipal de Administracao, durante 12 dias do mes de Janeiro de 2019.",Sim,CI,eventuais auxiliar gerais administracao dias,eventual auxiliar geral administracao dia
4,"servicos eventuais prestados como Vigia para atender a Secretaria Municipal de Administracao, durante 10 dias do mes de Janeiro de 2019.",Sim,CI,eventuais vigia administracao dias,eventual vigia administracao dia
5,"servicos eventuais prestados para atender a Secretaria Municipal de Agricultura na qualidade de Marceneiro na confeccao de uma carroceria em madeira de lei para o caminhao desta Secretaria, durante o mes de Janeiro de 2019.",Sim,CI,eventuais agricultura qualidade marceneiro confeccao carroceria madeira lei caminhao desta,eventual agricultura qualidade marceneiro confeccao carrocerio madeiro lei caminhao de este
6,"servicos prestado na confeccao de um portao de vergalhao medindo 2,22m por 2,52m para o Mercado Municipa/Secretaria Municipal de Agricultura, no mes de Janeiro de 2018.",Sim,CI,prestado confeccao portao vergalhao medindo mercado municipa agricultura,prestar confeccao Portao vergalhao medir mercado municipa agricultura
7,"servicos eventuais prestados para atender a Secretaria Municipal de Obras e Transportes, na qualidade de Vigia Diurno durante 25 dias do mes de Janeiro de 2018.",Sim,CI,eventuais obras transportes qualidade vigia diurno dias,eventual obra transporte qualidade vigia diurno dia
8,"servicos eventuais prestados para atender a Secretaria Municipal de Administracao e Financas, com manutencao de intalacao eletricas no predio da Prefeitura Municipal, no mes mde Janeiro de 2019.",Sim,CI,eventuais administracao financas manutencao intalacao eletricas predio prefeitura mde,eventual administracao financo manutencao intalacao eletrica predio prefeitura mde
9,"servicos eventuais prestados como Vigia Noturno para atender a Secretaria Municipal de Administracao, durante o mes de Janeiro de 2019.",Sim,CI,eventuais vigia noturno administracao,eventual vigia noturno administracao


## 3. Comparação de desempenho (texto original, texto limpo, texto lematizado)


In [ ]:
# Dataframe de teste para determinação das melhores técnicas de processamento de texto e melhores hyperparâmetros.
# O método sample estratifica o dataframe proporcionalmente às classes.
df_sample = df.sample(8000, random_state=42)

In [ ]:
# Define o pipeline incluindo o extrator de features do texto e um classificador
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('classifier', SGDClassifier(random_state=42))
])

In [ ]:
formatos_texto = ['descricao', 'descricao_limpa', 'descricao_lematizada']
df_desempenho = pd.DataFrame(formatos_texto, columns=['formato_texto'])
cv_scores = []
accuracies = []
f1_scores = []

for formato_texto in formatos_texto:

    # Separação entre variáveis preditoras e alvo e divisão em treino e teste
    X = df_sample[formato_texto]
    y = df_sample['categoria']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # Ajuste do modelo
    pipeline.fit(X_train, y_train)

    # Calcula as principais métricas de avaliação do modelo
    cv_scores.append(cross_val_score(pipeline, X_train, y_train, cv=5).mean())
    accuracies.append(accuracy_score(y_test, pipeline.predict(X_test)))
    f1_scores.append(f1_score(y_test, pipeline.predict(X_test), average='weighted'))

df_desempenho['cros_val_score'] = cv_scores
df_desempenho['accuracy'] = accuracies
df_desempenho['f1_score'] = f1_scores
df_desempenho

,formato_texto,cros_val_score,accuracy,f1_score
0,descricao,0.991071,0.988333,0.987897
1,descricao_limpa,0.989643,0.987917,0.987555
2,descricao_lematizada,0.989107,0.990833,0.990545


As métricas estão muito próximas para todos os formatos de texto, poderia utilizar qualquer uma delas. Contudo, no treinamento e otimização dos modelos, a descricao_lematizada teve melhor desempenho no tempo de execução e no número de iterações (no RandomForest, a descricao original, sem formatação, não convergiu com max_iter=1000).

## 4. Exportação para json

In [ ]:
# Exporta dataframe para json
df.to_json('/content/drive/My Drive/projeto classificacao de lancamento/dados/despesa liquidada - base completa - texto processado.json')